Here are some **easy to medium-level debugging and error analysis questions** for TensorFlow, along with code examples and explanations.  

---

### **1. Debugging Shape Mismatch in Model Input**  
**Question:**  
The following code tries to train a simple neural network but runs into a shape mismatch error. Fix it.  

```python
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

# Dummy data
X_train = tf.random.normal((100,))  # 1D tensor instead of 2D
y_train = tf.random.uniform((100,), maxval=2, dtype=tf.int32)

# Define model
model = Sequential([
    Dense(10, activation='relu', input_shape=(10,)),  # Input shape issue
    Dense(1, activation='sigmoid')
])

# Compile and train
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=5)
```

**Solution:**  
The issue is that `X_train` is **1D**, but the model expects a **2D input with shape (None, 10)**. We need to reshape `X_train` to `(100, 10)`.  

```python
X_train = tf.random.normal((100, 10))  # Now it's 2D
```

---

### **2. Handling NaNs in Training Data**  
**Question:**  
You notice that your loss becomes `NaN` during training. How do you debug the issue?  

```python
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense

# Create a dataset with NaNs
X_train = np.random.randn(100, 5)
y_train = np.random.randint(0, 2, size=(100,))

# Inject NaN values
X_train[0, 0] = np.nan

# Build simple model
model = tf.keras.Sequential([
    Dense(10, activation='relu', input_shape=(5,)),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train model
model.fit(X_train, y_train, epochs=5)
```

**Solution:**  
To debug NaNs in the dataset, use `np.isnan()` to check for missing values.  

```python
print(np.isnan(X_train).any())  # True, indicates NaNs are present
```

Fix it by replacing NaNs:  

```python
X_train = np.nan_to_num(X_train)  # Replace NaNs with 0
```

---

### **3. Fixing Incorrect Loss Function for Multi-Class Classification**  
**Question:**  
The following code trains a model for multi-class classification but throws an error. Fix it.  

```python
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

# Dummy data
X_train = tf.random.normal((100, 10))
y_train = tf.random.uniform((100,), maxval=3, dtype=tf.int32)  # 3 classes

# Model
model = Sequential([
    Dense(10, activation='relu', input_shape=(10,)),
    Dense(3, activation='softmax')  # 3 output classes
])

# Compile model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])  # Wrong loss function
model.fit(X_train, y_train, epochs=5)
```

**Solution:**  
Since this is a multi-class classification problem, we should use `SparseCategoricalCrossentropy` instead of `mean_squared_error`.  

```python
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
```

### **Comparison Table** (BR)
| Criterion | Binary Cross-Entropy | Sparse Categorical Cross-Entropy |
|-----------|----------------------|---------------------------------|
| **Number of Classes** | 2 (binary) or multi-label | More than 2 (multi-class) |
| **Label Format** | Single 0/1 value per sample | Integer class labels (e.g., `0,1,2,...`) |
| **Model Output** | `sigmoid` activation, single neuron | `softmax` activation, neurons = number of classes |
| **Example Task** | Spam detection, disease prediction | Image classification, sentiment analysis with >2 classes |

---

### **Common Mistakes**
❌ Using `binary_crossentropy` for multi-class classification  
❌ Using `sparse_categorical_crossentropy` with one-hot encoded labels (instead, use `categorical_crossentropy`)  
❌ Forgetting to match the activation function with the correct loss function (`sigmoid` for BCE, `softmax` for SCE)

---

### **4. Handling Exploding Gradients in Deep Networks**  
**Question:**  
Your deep neural network performs poorly because of **exploding gradients**. How do you fix it?  

```python
import tensorflow as tf
from tensorflow.keras.layers import Dense

# Model with deep architecture
model = tf.keras.Sequential([
    Dense(512, activation='relu', input_shape=(20,)),
    Dense(512, activation='relu'),
    Dense(512, activation='relu'),
    Dense(512, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
```

**Solution:**  
- Use **gradient clipping** to prevent extreme updates.  

```python
optimizer = tf.keras.optimizers.Adam(clipnorm=1.0)  # Clips gradients
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
```

- Try **Batch Normalization** or **smaller learning rate**.  

---

### **5. Fixing Incorrect Accuracy Calculation in Multi-Label Classification**  
**Question:**  
You are working on a **multi-label classification** task where each sample can belong to multiple classes. The following code uses `accuracy` as a metric, but it produces misleading results. Fix it.  

```python
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

# Dummy data
X_train = tf.random.normal((100, 10))
y_train = tf.random.uniform((100, 3), maxval=2, dtype=tf.int32)  # Multi-label (each sample has multiple 0/1 labels)

# Model
model = Sequential([
    Dense(10, activation='relu', input_shape=(10,)),
    Dense(3, activation='sigmoid')  # Multi-label output
])

# Compile model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=5)
```

**Solution:**  
- Use **binary cross-entropy** instead of sparse categorical cross-entropy, since each class is **independent**.  
- Use **`AUC`** or **`F1-score`** instead of `accuracy` for multi-label problems.  

```python
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['AUC'])
```

---

### **Bonus: Detecting Silent Errors in Model Training**  
If you want to **detect silent numerical issues** during training, use:  

```python
tf.debugging.enable_check_numerics()  # Detects NaNs or Infs in TensorFlow tensors
```

---

### **Summary of Debugging Issues and Fixes**
| Issue | Fix |
|---------------------------|----------------------------------------|
| Shape mismatch error | Ensure correct input shape for training data |
| NaNs in dataset | Check with `np.isnan()` and replace using `np.nan_to_num()` |
| Wrong loss function for classification | Use `sparse_categorical_crossentropy` for integer labels, `categorical_crossentropy` for one-hot labels |
| Exploding gradients | Use **gradient clipping**, **batch normalization**, or reduce learning rate |
| Wrong metric for multi-label classification | Use `binary_crossentropy` and **AUC** instead of `accuracy` |

Would you like more debugging examples? 😊

Here are some **easy to medium debugging and error analysis questions** related to TensorFlow code. These examples will help you practice identifying and fixing common issues in machine learning pipelines.

---

### **Question 1: Debugging a Shape Mismatch**
You are training a model, but you encounter the following error:
```
ValueError: Shapes (None, 1) and (None, 10) are incompatible
```

#### **Code**:
```python
import tensorflow as tf
from tensorflow.keras import layers

# Define the model
model = tf.keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(32,)),
    layers.Dense(32, activation='relu'),
    layers.Dense(10, activation='softmax')  # Output layer with 10 units
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Dummy data
X_train = tf.random.normal((1000, 32))
y_train = tf.random.uniform((1000,), maxval=10, dtype=tf.int32)

# Train the model
model.fit(X_train, y_train, epochs=10)
```

#### **Issue**:
The error occurs because `sparse_categorical_crossentropy` expects `y_train` to be integer labels (shape `(None,)`), but the output layer produces a probability distribution over 10 classes (shape `(None, 10)`). The shapes are incompatible.

#### **Fix**:
Ensure the output layer matches the loss function. Since `y_train` is integer labels, the output layer should have 10 units with a `softmax` activation, and the loss should remain `sparse_categorical_crossentropy`.

```python
# No changes needed; the code is correct.
```

---

### **Question 2: Debugging a NaN Loss**
You are training a model, but the loss becomes `NaN` after a few epochs. Debug the issue.

#### **Code**:
```python
import tensorflow as tf
from tensorflow.keras import layers

# Define the model
model = tf.keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(32,)),
    layers.Dense(32, activation='relu'),
    layers.Dense(1, activation='sigmoid')  # Output layer for binary classification
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Dummy data
X_train = tf.random.normal((1000, 32))
y_train = tf.random.uniform((1000,), maxval=2, dtype=tf.int32)

# Train the model
model.fit(X_train, y_train, epochs=10)
```

#### **Issue**:
The issue is likely due to the data. If `y_train` contains values other than `0` or `1`, the `binary_crossentropy` loss can produce `NaN` values.

#### **Fix**:
Ensure `y_train` contains only `0` or `1` for binary classification.

```python
# Convert y_train to binary labels (0 or 1)
y_train = tf.cast(y_train, dtype=tf.float32)
y_train = tf.where(y_train > 0, 1.0, 0.0)

# Train the model
model.fit(X_train, y_train, epochs=10)
```

---

### **Question 3: Debugging a Slow Training Process**
You notice that the training process is very slow. Identify potential issues and suggest improvements.

#### **Code**:
```python
import tensorflow as tf
from tensorflow.keras import layers

# Define the model
model = tf.keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(1000,)),
    layers.Dense(32, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Dummy data
X_train = tf.random.normal((10000, 1000))
y_train = tf.random.uniform((10000,), maxval=2, dtype=tf.int32)

# Train the model
model.fit(X_train, y_train, epochs=10)
```

#### **Issue**:
The input data has a very high dimensionality (`1000` features), which can make training slow. Additionally, the model architecture might be too simple for the data.

#### **Fix**:
1. **Reduce Dimensionality**: Use dimensionality reduction techniques like PCA or feature selection.
2. **Increase Model Capacity**: Add more layers or units to the model.
3. **Use Batch Training**: Ensure the data is batched properly.

```python
# Example: Increase model capacity
model = tf.keras.Sequential([
    layers.Dense(256, activation='relu', input_shape=(1000,)),
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

# Compile and train the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=10, batch_size=32)
```

---

### **Question 4: Debugging Overfitting**
You notice that the model performs well on the training data but poorly on the validation data. Debug the issue.

#### **Code**:
```python
import tensorflow as tf
from tensorflow.keras import layers

# Define the model
model = tf.keras.Sequential([
    layers.Dense(128, activation='relu', input_shape=(32,)),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Dummy data
X_train = tf.random.normal((1000, 32))
y_train = tf.random.uniform((1000,), maxval=2, dtype=tf.int32)
X_val = tf.random.normal((200, 32))
y_val = tf.random.uniform((200,), maxval=2, dtype=tf.int32)

# Train the model
model.fit(X_train, y_train, epochs=50, validation_data=(X_val, y_val))
```

#### **Issue**:
The model is overfitting because it is too complex relative to the amount of training data.

#### **Fix**:
1. **Add Regularization**: Use dropout or L2 regularization.
2. **Reduce Model Complexity**: Use fewer layers or units.
3. **Early Stopping**: Stop training when validation performance stops improving.

```python
# Add dropout and early stopping
model = tf.keras.Sequential([
    layers.Dense(128, activation='relu', input_shape=(32,)),
    layers.Dropout(0.5),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

# Add early stopping callback
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=5, restore_best_weights=True
)

# Compile and train the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=50, validation_data=(X_val, y_val), callbacks=[early_stopping])
```

---

### **Question 5: Debugging a Data Pipeline Issue**
You are using a `tf.data.Dataset` pipeline, but the model is not training. Debug the issue.

#### **Code**:
```python
import tensorflow as tf

# Create a dataset
dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
dataset = dataset.batch(32)

# Define the model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(32,)),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(dataset, epochs=10)
```

#### **Issue**:
The dataset is not being shuffled, which can lead to poor training performance.

#### **Fix**:
Shuffle the dataset before batching.

```python
# Shuffle the dataset
dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
dataset = dataset.shuffle(buffer_size=1000).batch(32)

# Train the model
model.fit(dataset, epochs=10)
```

---

### **Summary**
These examples cover common debugging scenarios in TensorFlow, including:
1. Shape mismatches.
2. NaN losses.
3. Slow training.
4. Overfitting.
5. Data pipeline issues.

Practicing these will help you develop strong debugging and error analysis skills for TensorFlow code.